In [3]:
import os
import json
from datetime import datetime
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj=SentimentIntensityAnalyzer()
GLASSDOOR_DATA_DIR = 'glassdoor/data/'
INDEED_DATA_DIR = 'indeed/data/'
glassdoor_files=os.listdir(GLASSDOOR_DATA_DIR)
print(glassdoor_files)
indeed_files=os.listdir(INDEED_DATA_DIR)
print(indeed_files)

['Amazon.txt', 'Berkshire-Hathaway.txt', 'Blackrock.txt', 'Cisco-Systems.txt', 'Facebook.txt', 'Johnson-and-Johnson.txt', 'Microsoft.txt', 'Sony.txt']
['Adidas.txt', 'Amazon.com.txt', 'Berkshire-Hathaway.txt', 'Microsoft.txt']


In [31]:
all_data=[]
#with open(GLASSDOOR_DATA_DIR+'Amazon.txt',encoding='utf-8') as f:
with open('glassdoor/Microsoft.txt',encoding='utf-8') as f:
    for line in f:
        data=json.loads(line)
        data['rating']=float(data['rating'])
        sTemp=data['employeeStatus'].split(',')
        if len(sTemp)>1:
            data['employmentStatus']=sTemp[0].strip()
            data['employmentTenure']=sTemp[1].strip()
        else:
            data['employmentStatus']=data['employeeStatus'].strip()
            data['employmentTenure']=''
        data['company']='Amazon'
        data['datePosted']=datetime.strptime(data['authorInfo'].split('-')[0].strip(),'%d %b %Y')
        data['authorProfession']=data['authorInfo'].split('-')[1].strip()
        data.pop('employeeStatus',None)
        data.pop('adviceManagement',None)
        data.pop('authorInfo',None)
        all_data.append(data)
df_glassdoor=pd.DataFrame(all_data)
print(df_glassdoor.shape)
df_glassdoor.head()

(3690, 9)


,rating,pros,cons,authorLocation,employmentStatus,employmentTenure,company,datePosted,authorProfession
0,5.0,Nice Vibe and Talented Colleagues yay,Too far away from city center,Hong Kong,Former Employee,more than 1 year,Amazon,2021-07-13,Microsoft Student Partner
1,3.0,Relatively good compensation and benefit. Stoc...,Very hierarchical. Not a lot of PMs know how t...,,Current Employee,more than 1 year,Amazon,2021-07-06,Designer
2,4.0,Good opportunity to grow and expand in company...,Totally localised environment and will require...,Hong Kong,Former Employee,less than 1 year,Amazon,2021-06-19,Anonymous Employee
3,5.0,Flexible Hours and working location\r\nDiversi...,Less interaction between colleagues\r\nWide jo...,Hong Kong,Current Employee,more than 1 year,Amazon,2021-05-13,Customer Success Account Manager
4,5.0,Employer is great and nice atmosphere,Nothing really. It is a nice company.,Hong Kong,Current Employee,,Amazon,2021-05-14,Graphic Designer


In [27]:
def apply_sentiment(text):
    sentiment_dict=sid_obj.polarity_scores(text)
    return sentiment_dict['compound']

def search_reviews(sWords, matchPhrase=True, ignoreCase=True):
    if matchPhrase==False:
        sWords='|'.join([word for word in sWords.split()])
    df_subset=df[(df['pros'].str.contains(sWords,case=ignoreCase)==True)|(df['cons'].str.contains(sWords,case=ignoreCase)==True)]
    return df_subset

In [28]:
df_glassdoor['proSentiment']=df_glassdoor['pros'].apply(lambda x:apply_sentiment(x))
df_glassdoor['conSentiment']=df_glassdoor['cons'].apply(lambda x:apply_sentiment(x))
df.head()

,rating,employeeStatus,pros,cons,adviceManagement,authorInfo,authorLocation,employmentStatus,employmentTenure,company,datePosted,authorProfession
0,5.0,"Former Employee, more than 1 year",Nice Vibe and Talented Colleagues yay,Too far away from city center,,13 Jul 2021 - Microsoft Student Partner,Hong Kong,Former Employee,more than 1 year,Amazon,2021-07-13,Microsoft Student Partner
1,3.0,"Current Employee, more than 1 year",Relatively good compensation and benefit. Stoc...,Very hierarchical. Not a lot of PMs know how t...,,6 Jul 2021 - Designer,,Current Employee,more than 1 year,Amazon,2021-07-06,Designer
2,4.0,"Former Employee, less than 1 year",Good opportunity to grow and expand in company...,Totally localised environment and will require...,,19 Jun 2021 - Anonymous Employee,Hong Kong,Former Employee,less than 1 year,Amazon,2021-06-19,Anonymous Employee
3,5.0,"Current Employee, more than 1 year",Flexible Hours and working location\r\nDiversi...,Less interaction between colleagues\r\nWide jo...,,13 May 2021 - Customer Success Account Manager,Hong Kong,Current Employee,more than 1 year,Amazon,2021-05-13,Customer Success Account Manager
4,5.0,Current Employee,Employer is great and nice atmosphere,Nothing really. It is a nice company.,,14 May 2021 - Graphic Designer,Hong Kong,Current Employee,,Amazon,2021-05-14,Graphic Designer


In [14]:
df_temp=search_reviews('pan')
print(df_temp.shape)
df_temp.head()

(12569, 11)


,rating,pros,cons,adviceManagement,authorInfo,authorLocation,company,datePosted,authorProfession,proSentiment,conSentiment
0,5.0,Love the perks of the company,No Cons as such in mind,,20 Jun 2021 - Content Analyst,Bangalore,Amazon,2021-06-20,Content Analyst,0.6369,-0.2960
6,4.0,I've had the chance to launch multiple amazon ...,Compensation lower for internal promos,,19 Jun 2021 - Senior Operations Manager,"Dallas, TX",Amazon,2021-06-19,Senior Operations Manager,0.7269,-0.2960
19,4.0,"Set schedule, 3 day weekend, good benefits.",Pay is not there considering the size of the c...,,18 Jun 2021 - Wharehouse Associate,"Middletown, DE",Amazon,2021-06-18,Wharehouse Associate,0.6705,-0.1027
29,4.0,"Set schedule, 3 day weekend, good benefits.",Pay is not there considering the size of the c...,,18 Jun 2021 - Wharehouse Associate,"Middletown, DE",Amazon,2021-06-18,Wharehouse Associate,0.6705,-0.1027
32,5.0,I love the inclusion I feel in this company. A...,Less overtime opportunities could be the only ...,,18 Jun 2021 - Customer Service Representative,Pune,Amazon,2021-06-18,Customer Service Representative,0.5719,0.3230


In [15]:
sample_size=5
temp=df.sample(sample_size)
for i in range(sample_size):
    print(temp.iloc[i]['rating'])
    print(temp.iloc[i]['pros'])
    print(temp.iloc[i]['proSentiment'])
    print(temp.iloc[i]['cons'])
    print(temp.iloc[i]['conSentiment'])
    print("________________________________")

4.0
Great culture, great people, interesting problems, demanding work. Lots of opportunities for learning.
0.872
Amazon has a leadership principle of frugality. This translates to people being cheap, corporate sponsored events being few and far between.
0.1779
________________________________
5.0
fast paced 
fun environment 
benefits
0.7096
overtime
repetitive work
physically hard
can be boring sometimes
-0.5719
________________________________
4.0
Very Competitve, all round development,
0.0
Too much work load on employee
0.0
________________________________
1.0
There are no pros zero
-0.296
Terrible management, petty management, allow the lazy to be lazy, and push the hard working harder.
-0.8126
________________________________
5.0
It looks an wonderful environment to enjoy friends an we have to know the value of money
0.9081
I have interest to work in such a company because they have providing a good opportunity to earn our self
0.8271
________________________________


In [7]:
from gensim.models import Word2Vec
model=Word2Vec.load('word2vec.model_50_3_1')

In [36]:
word_dict={}
nidhi_words=['Gender','Asian','Indian','White','Male','Female',
             'Latin','Diversity','Equity',
             'Inclusion','Stereotype','Equality']
for word in nidhi_words:
    items=model.wv.most_similar(word.lower(),topn=5)
    for item in items:
        word_dict[item[0]]=1
len(word_dict)

54

In [37]:
model.wv.most_similar('female',topn=5)

[('male', 0.795523464679718),
 ('seasoned', 0.7916832566261292),
 ('veteran', 0.7370079755783081),
 ('incapable', 0.7367067337036133),
 ('minorities', 0.7150292992591858)]

In [38]:
word_dict

{'racial': 1,
 'fairness': 1,
 'discrimination': 1,
 'privacy': 1,
 'compliance': 1,
 'stuffed': 1,
 'predominantly': 1,
 'spanish': 1,
 'submissive': 1,
 'pan': 1,
 'male': 1,
 'chinese': 1,
 'elite': 1,
 'australian': 1,
 'pms': 1,
 'badges': 1,
 'men': 1,
 'golden': 1,
 'males': 1,
 'female': 1,
 'veteran': 1,
 'white': 1,
 'seasoned': 1,
 'incapable': 1,
 'minorities': 1,
 'emea': 1,
 'blr': 1,
 'de': 1,
 'fortress': 1,
 'otc': 1,
 'inclusion': 1,
 'teamwork': 1,
 'empowerment': 1,
 'transparency': 1,
 'professionalism': 1,
 'awards': 1,
 'valuation': 1,
 'grants': 1,
 'backloaded': 1,
 'offerings': 1,
 'openness': 1,
 'diversity': 1,
 'cultural': 1,
 'inexcusable': 1,
 'investigators': 1,
 'slowest': 1,
 'hightech': 1,
 'beauty': 1,
 'brilliance': 1,
 'honesty': 1,
 'sexism': 1,
 'payroll': 1,
 'messaging': 1,
 'piss-poor': 1}

In [78]:
all_data=[]
with open(INDEED_DATA_DIR+'Microsoft.txt',encoding='utf-8') as f:
    for line in f:
        data=json.loads(line.replace('Tripoli - Libya','Tripoli, Libya'))
        data['rating']=float(data['rating'])
        sectionData=data['authorInfo'].split('-')
        if len(sectionData)>3:
            temp=data['authorInfo'].split('-')
            sectionData=[' '.join([temp[i] for i in range(0,len(temp)-2)])]
            for i in range(len(temp)-2,len(temp)):
                sectionData.append(temp[i])
        if '(' in sectionData[0]:
            temp2=sectionData[0].split('(')
            data['authorProfession']=temp2[0].strip()
            data['employeeStatus']=temp2[1].strip()[:-1]
        else:
            data['authorProfession']=sectionData[0].strip()
        data['authorLocation']=sectionData[1].strip()
        data['datePosted']=datetime.strptime(sectionData[2].strip(),'%B %d, %Y')
        all_data.append(data)
df=pd.DataFrame(all_data)
print(df.shape)
df.head()

(6547, 9)


,rating,desc,pros,cons,authorInfo,authorProfession,employeeStatus,authorLocation,datePosted
0,5.0,it was a good experience that helped me learn ...,free kaunches,,"dev (Former Employee) - California - July 14, ...",dev,Former Employee,California,2021-07-14
1,5.0,I like the people there I suppose and the work...,,,"Consultant (Current Employee) - New York, NY -...",Consultant,Current Employee,"New York, NY",2021-07-13
2,5.0,pay is pretty low if you are working in the da...,,,Data Center Technician (Former Employee) - San...,Data Center Technician,Former Employee,"Santa Clara, CA",2021-07-11
3,5.0,Great work-life balance (it's a big company so...,,,"Developer (Current Employee) - Redmond, WA - J...",Developer,Current Employee,"Redmond, WA",2021-07-09
4,5.0,Working with Microsoft is bit lovely and amazi...,Company meetings on Lunch,Excellent health care,Partner Account Manager Western Region (Former...,Partner Account Manager Western Region,Former Employee,Jeddah,2021-06-26
